In [1]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# 1. Tách từ

## Sử dụng thuật toán Longest Matching

In [3]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [4]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [5]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [6]:
bi_grams = load_n_grams('sources/bi_grams.txt')
tri_grams = load_n_grams('sources/tri_grams.txt')
longest_matching('nhưng sự thực hiện vẫn còn chưa phù hợp', bi_grams, tri_grams)

['nhưng', 'sự_thực', 'hiện', 'vẫn', 'còn', 'chưa', 'phù_hợp']

## Sử dụng thư viện VnCoreNLP

In [7]:
from vncorenlp import VnCoreNLP
client = VnCoreNLP(address="http://127.0.0.1", port=9001)
word_list = client.tokenize('nhưng sự thực hiện vẫn còn chưa phù hợp')
word_list[0]

['nhưng', 'sự', 'thực_hiện', 'vẫn', 'còn', 'chưa', 'phù_hợp']

## Tạo ngữ liệu

In [8]:
sentences = open('sources/sentences.txt', encoding='utf-8').readlines()
print('Số lượng câu:', len(sentences))
sentences[0:2]

Số lượng câu: 60


['Pha lập công trên đã giúp Rashford giải hạn bàn thắng tại sân Old Trafford kéo dài 845 phút .\n',
 'Với 3 điểm có được trong trận đấu này , Quỷ đỏ đã leo lên vị trí thứ 2 trên bảng xếp hạng Premier League với 30 điểm , chỉ kém đội đầu bảng Liverpool 2 điểm .\n']

In [9]:
with open('sources/tokenize.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        word_list = client.tokenize(sentence)
        for word in word_list[0]: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')

In [10]:
tokenize_corpus = open('sources/tokenize.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(tokenize_corpus))
tokenize_corpus[0:5]

Số lượng từ: 1006


['Pha\n', 'lập_công\n', 'trên\n', 'đã\n', 'giúp\n']

# 2. Đọc dữ liệu

In [11]:
def preprocess(vocabs, path):
    data = []
    file = open(path, encoding='utf-8').readlines()
    
    for index, word in enumerate(file):
        if not word.split():
            word = '--n--'
            data.append(word)
            continue
        elif word.lower().strip() not in vocabs:
            word = '--unk--'
            data.append(word)
            continue
        data.append(word.strip())
    return data

In [12]:
vocabs = open('sources/vocabs.txt', encoding='utf-8').read().split('\n')
print('Số lượng từ vựng:', len(vocabs))
vocabs[0:5]

Số lượng từ vựng: 54934


['a', 'A', 'a_dua', 'a_ha', 'a_lô']

In [13]:
test_words = preprocess(vocabs, 'test.txt')
print('Số lượng từ trong tập test:', len(test_words))
test_words[0:5]

Số lượng từ trong tập test: 119


['Những', 'ngày', 'đẹp_đẽ', 'ấy', ',']

In [14]:
gold_corpus = open('gold.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập gold:', len(gold_corpus))
gold_corpus[0:5]

Số lượng từ trong tập gold: 119


['Những\tL\n', 'ngày\tN\n', 'đẹp_đẽ\tA\n', 'ấy\tP\n', ',\tCH\n']

In [15]:
train_corpus = open('train.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập train:', len(train_corpus))
train_corpus[0:5]

Số lượng từ trong tập train: 886


['Pha\tNp\n', 'lập_công\tV\n', 'trên\tE\n', 'đã\tR\n', 'giúp\tV\n']

# 3. Parts of Speech Tagging

## Training

In [16]:
def seperate_word_tag(word_tag, vocabs): 
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
        return word, tag
    else:
        word, tag = word_tag.split()
        if word.lower() not in vocabs: word = '--unk--'
        return word, tag
    return None 

In [17]:
def create_dictionaries(train_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    i = 0 
    
    for word_tag in train_corpus:
        i += 1
        word, tag = seperate_word_tag(word_tag, vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return emission_counts, transition_counts, tag_counts

In [18]:
emission_counts, transition_counts, tag_counts = create_dictionaries(train_corpus, vocabs)
states = sorted(tag_counts.keys())
print('Số lượng nhãn:', len(states))
print(states)

Số lượng nhãn: 17
['--s--', 'A', 'C', 'CH', 'Cc', 'E', 'L', 'M', 'N', 'Nc', 'Np', 'Nu', 'P', 'R', 'T', 'V', 'Z']


In [19]:
print("Transition examples: ")
for example in list(transition_counts.items())[:3]:
    print(example)

Transition examples: 
(('--s--', 'Np'), 19)
(('Np', 'V'), 13)
(('V', 'E'), 26)


In [20]:
print("Emission examples: ")
for example in list(emission_counts.items())[200:203]:
    print (example)

Emission examples: 
(('V', 'thông_báo'), 1)
(('V', 'hết'), 1)
(('N', 'tinh_thần'), 1)


## Testing

In [21]:
def predict_pos(test_words, gold_corpus, emission_counts, vocabs, states):
    num_correct = 0
    all_words = set(emission_counts.keys())
    total = len(gold_corpus)
    
    for word, gold_tuple in zip(test_words, gold_corpus): 
        gold_tuple_list = gold_tuple.split()
        if len(gold_tuple_list) != 2: continue
        else: true_label = gold_tuple_list[1]
    
        count_final = 0
        pos_final = ''
        if word.lower() not in vocabs: continue
        
        for pos in states:
            if (pos, word) not in emission_counts: continue
            count = emission_counts[(pos, word)]
            
            if count > count_final:
                count_final = count
                pos_final = pos
                    
        if pos_final == true_label: num_correct += 1
    accuracy = num_correct / total
    return accuracy

In [22]:
accuracy = predict_pos(test_words, gold_corpus, emission_counts, vocabs, states)
print('Độ chính xác của dự đoán:', accuracy)

Độ chính xác của dự đoán: 0.4789915966386555


# 4. Ma trận xác suất Hidden Markov

## Ma trận chuyển tiếp 'A' (transition matrix)

In [23]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags, num_tags))
    trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i],all_tags[j])
            if key in transition_counts: count = transition_counts[key]
                
            count_prev_tag = tag_counts[all_tags[i]]
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)
    return A

In [24]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)
df = pd.DataFrame(
    A[5:10, 5:10], 
    index = states[5:10], 
    columns = states[5:10]
)
df.head()

,E,L,M,N,Nc
E,0.000015,0.075753,0.075753,0.393853,0.045458
L,0.000067,0.000067,0.000067,0.732570,0.000067
M,0.083316,0.000042,0.124953,0.458051,0.000042
N,0.099053,0.000005,0.018871,0.212252,0.009438
Nc,0.000062,0.000062,0.000062,0.187363,0.000062


## Ma trận phát xạ 'B' (emission matrix)

In [25]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocabs):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocabs)
    
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (all_tags[i], vocabs[j])
            if key in emission_counts.keys(): count = emission_counts[key]
                
            count_tag = tag_counts[all_tags[i]]
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)
    return B

In [26]:
cidx  = ['thông_báo', 'hạt_nhân', 'tinh_thần', 'lập_công', 'vị_trí']
rvals = ['N', 'V', 'CH', 'Cc', 'E', 'L']
cols = [vocabs.index(a) for a in cidx]
rows = [states.index(a) for a in rvals]

B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocabs))
df = pd.DataFrame(B[np.ix_(rows, cols)], index=rvals, columns=cidx)
df.head()

,thông_báo,hạt_nhân,tinh_thần,lập_công,vị_trí
N,0.000004,0.000004,0.003750,0.000004,0.000004
V,0.003654,0.000004,0.000004,0.003654,0.000004
CH,0.000007,0.000007,0.000007,0.000007,0.000007
Cc,0.000015,0.000015,0.000015,0.000015,0.000015
E,0.000008,0.000008,0.000008,0.000008,0.000008


# 5. Thuật toán Viterbi

## Bước Initialization

In [27]:
def viterbi_initialize(states, tag_counts, A, B, corpus, vocabs):
    num_tags = len(tag_counts)
    s_idx = states.index('--s--')
    
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    for i in range(num_tags):
        if A[s_idx,i] == 0: best_probs[i, 0] = float('-inf')
        else: 
            index = vocabs.index(corpus[0].lower())
            best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, index])
    return best_probs, best_paths

In [28]:
best_probs, best_paths = viterbi_initialize(states, tag_counts, A, B, test_words, vocabs)
print('best_probs[0, 0]:', best_probs[0, 0]) 
print('best_paths[2, 3]:', best_paths[2, 3])

best_probs[0, 0]: -22.351433816984247
best_paths[2, 3]: 0


## Bước Forward

In [29]:
def viterbi_forward(A, B, corpus, best_probs, best_paths, vocabs):
    num_tags = best_probs.shape[0]
    
    for i in range(1, len(corpus)): 
        if i % 5000 == 0: print(f'Processed {i} words...')
            
        for j in range(num_tags):
            best_prob_i = float('-inf')
            best_path_i = None
            
            for k in range(num_tags):
                index = vocabs.index(corpus[i].lower())
                prob = best_probs[k, i - 1] + math.log(A[k, j]) + math.log(B[j, index])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
                    
            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i
            
    return best_probs, best_paths

In [30]:
best_probs, best_paths = viterbi_forward(A, B, test_words, best_probs, best_paths, vocabs)
print('best_probs[0, 1]:', best_probs[0, 1]) 
print('best_probs[0, 4]:', best_probs[0, 4])

best_probs[0, 1]: -28.208223583028193
best_probs[0, 4]: -52.447189842755186


## Bước Backward

In [31]:
def viterbi_backward(best_probs, best_paths, corpus, states):
    m = best_paths.shape[1] 
    z = [None] * m
    
    num_tags = best_probs.shape[0]
    best_prob_for_last_word = float('-inf')
    pred = [None] * m
    
    for k in range(num_tags):
        if best_probs[k,-1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, -1]
            z[m - 1] = k
            
    pred[m - 1] = states[k]
    for i in range(len(corpus) - 1, -1, -1):
        pos_tag = best_paths[np.argmax(best_probs[:, i]), i]
        z[i - 1] = best_paths[pos_tag, i]
        pred[i - 1] = states[pos_tag]
    return pred

In [32]:
pred = viterbi_backward(best_probs, best_paths, test_words, states)
m = len(pred)

print(f'Dự đoán cho pred[-7:{m - 1}]:')
print(test_words[-7:m-1])
print(pred[-7:m-1])

print('Dự đoán cho pred[0:7]:')
print(test_words[0:7])
print(pred[0:7])

Dự đoán cho pred[-7:118]:
['là', 'công_cụ', 'không_thể', 'đảo_ngược', 'trong', 'cuộc_sống']
['V', 'P', 'R', 'V', 'E', 'N']
Dự đoán cho pred[0:7]:
['Những', 'ngày', 'đẹp_đẽ', 'ấy', ',', 'bố', 'luôn']
['L', 'N', 'CH', 'Nu', 'CH', 'P', 'R']


In [33]:
def predict_sentence(sentence):
    output = ''
    for word in sentence.split():
        if word not in test_words: word = '--unk--'
        index = test_words.index(word)
        output += f'{word}/{pred[index]} '
    return output.strip()

In [34]:
for sentence in sentences[50:]:
    word_list = client.tokenize(sentence)
    print(predict_sentence(' '.join(word_list[0])))

Những/L ngày/N đẹp_đẽ/CH ấy/Nu ,/CH bố/P luôn/R dành/V thời_gian/N để/E vui_vẻ/N với/E anh_em/Nc tôi/P ./CH
Trong/E khi/N chờ/E nước/N nóng/A ,/CH tôi/P pha/V rượu/N táo/CH nóng/A ./CH
Hàng/N trăm/M người/N đến/V dùng/V tiệc/CH tự/--s-- chọn/Np ở/E chỗ/CH mẹ/E những/L ngày/N Giáng_sinh/A ./CH
Năm/N nào/V cũng/R vậy/A hoặc/Cc --unk--/N --unk--/N ./CH
Tôi/P chưa/CH bao_giờ/--s-- thực_sự/Np --unk--/N mệt_mỏi/CH vì/--s-- nó/P ./CH
Tôi/P đã/R có_thể/V chống/P lại/R cảm_giác/V đó/V trong/E nhiều/A năm/N qua/V ./CH
Những/L năm/N gần/CH đây/--s-- tôi/P qua_lại/CH sân_bay/--s-- --unk--/N thường_xuyên/N ./CH
Trường/N tôi/P từng/CH nghiêm_cấm/--s-- sử_dụng/N điện_thoại/V trong/E lớp/N ./CH
Học_sinh/Np ngày_nay/R có_thể/V dễ_dàng/CH tiếp_cận/--s-- bài_học/Nc và/Cc phương_pháp/L giải/N bài_tập/A ./CH
Điện_thoại/Np thông_minh/R là/V công_cụ/P không_thể/R đảo_ngược/V trong/E cuộc_sống/N ./CH


## Đánh giá thuật toán

In [35]:
y_pred = []
y_true = []

for prediction, word_tag in zip(pred, gold_corpus):
    word_tag_tuple = word_tag.split()
    if len(word_tag_tuple) != 2: continue 

    word, tag = word_tag_tuple
    y_pred.append(prediction)
    y_true.append(tag)

In [36]:
from sklearn.metrics import classification_report
print('Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi:\n')
print(classification_report(y_pred, y_true))

Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi:

              precision    recall  f1-score   support

       --s--       0.00      0.00      0.00         8
           A       0.20      0.40      0.27         5
           C       0.00      0.00      0.00         0
          CH       0.92      0.52      0.67        21
          Cc       1.00      1.00      1.00         2
           E       0.88      0.78      0.82         9
           L       1.00      0.75      0.86         4
           M       1.00      1.00      1.00         1
           N       0.48      0.62      0.54        21
          Nc       0.33      0.50      0.40         2
          Np       0.00      0.00      0.00         5
          Nu       0.00      0.00      0.00         2
           P       0.50      0.70      0.58        10
           R       0.71      0.71      0.71         7
           V       0.45      0.69      0.55        13

    accuracy                           0.55       110
   macro avg     

# 6. So sánh kết quả với VnCoreNLP

In [37]:
y_pred_lib = []

for word_tag in gold_corpus:
    word_tag_tuple = word_tag.split()
    if len(word_tag_tuple) != 2: 
        print()
        continue 
        
    word, tag = word_tag_tuple
    if '_' not in word: pred = client.pos_tag(word)
    else: pred = client.pos_tag(word.replace('_', ' '))
    
    print(f'{word}/{pred[0][0][1]}', end=' ') 
    y_pred_lib.append(pred[0][0][1])

Những/L ngày/N đẹp_đẽ/A ấy/P ,/CH bố/N luôn/R dành/V thời_gian/N để/E vui_vẻ/A với/E anh_em/N tôi/P ./CH 
Trong/E khi/N chờ/V nước/N nóng/A ,/CH tôi/P pha/V rượu/N táo/V nóng/A ./CH 
Hàng/N trăm/M người/N đến/V dùng/V tiệc/N tự/P chọn/V ở/E chỗ/N mẹ/N những/L ngày/N Giáng_sinh/Np ./CH 
Năm/Np nào/P cũng/R vậy/P hoặc/Cc có_vẻ/X như_vậy/X ./CH 
Tôi/P chưa/R bao_giờ/P thực_sự/A cảm_thấy/V mệt_mỏi/A vì/E nó/P ./CH 
Tôi/P đã/R có_thể/R chống/V lại/R cảm_giác/N đó/P trong/E nhiều/A năm/N qua/V ./CH 
Những/L năm/N gần/A đây/P tôi/P qua_lại/V sân_bay/N Tân_Sơn_Nhất/Np thường_xuyên/A ./CH 
Trường/Np tôi/P từng/P nghiêm_cấm/V sử_dụng/V điện_thoại/N trong/E lớp/N ./CH 
Học_sinh/N ngày_nay/N có_thể/R dễ_dàng/A tiếp_cận/V bài_học/N và/Cc phương_pháp/N giải/N bài_tập/N ./CH 
Điện_thoại/N thông_minh/A là/V công_cụ/N không_thể/R đảo_ngược/V trong/E cuộc_sống/N ./CH 

In [38]:
print('Kết quả khi sử dụng thư viện VnCoreNLP:\n')
print(classification_report(y_pred_lib, y_true))

Kết quả khi sử dụng thư viện VnCoreNLP:

              precision    recall  f1-score   support

           A       0.90      0.82      0.86        11
           C       0.00      0.00      0.00         0
          CH       1.00      1.00      1.00        12
          Cc       1.00      1.00      1.00         2
           E       1.00      1.00      1.00         8
           L       1.00      1.00      1.00         3
           M       1.00      1.00      1.00         1
           N       0.85      0.82      0.84        28
          Nc       0.00      0.00      0.00         0
          Np       1.00      0.50      0.67         4
           P       1.00      0.93      0.97        15
           R       0.86      0.75      0.80         8
           V       0.65      0.81      0.72        16
           X       0.00      0.00      0.00         2

    accuracy                           0.85       110
   macro avg       0.73      0.69      0.70       110
weighted avg       0.87      0.85      